In [0]:
dados = [
    ("João", "Santos", "Brasileiro", "Estudante", "M", 500),
    ("Pedro", "Coelho", "Portugues", "Professor", "M", 2500),
    ("Krursa", "Zinovich", "Russa", "Medica", "F", 5000),
    ("Giani", "Alcomini", "Italiano", "Analista de sistemas", "M", 3000),
    ("Carol", "Smith", "Americano", "Arquiteta", "F", 2000),
    ("Peter", "Smith", "Americano", "Empresario", "M", 10000)]
colunas = ["PrimeiroNome", "UltimoNome", "Nacionalidade", "Profissao", "Genero", "Salario"]
dfparquet = spark.createDataFrame(dados, colunas)
dfparquet.show()

+------------+----------+-------------+--------------------+------+-------+
|PrimeiroNome|UltimoNome|Nacionalidade|           Profissao|Genero|Salario|
+------------+----------+-------------+--------------------+------+-------+
|        João|    Santos|   Brasileiro|           Estudante|     M|    500|
|       Pedro|    Coelho|    Portugues|           Professor|     M|   2500|
|      Krursa|  Zinovich|        Russa|              Medica|     F|   5000|
|       Giani|  Alcomini|     Italiano|Analista de sistemas|     M|   3000|
|       Carol|     Smith|    Americano|           Arquiteta|     F|   2000|
|       Peter|     Smith|    Americano|          Empresario|     M|  10000|
+------------+----------+-------------+--------------------+------+-------+



In [0]:
dfparquet.write.mode('overwrite').parquet("/dbfs/FileStore/tables/carga/usuarios")

In [0]:
dfleitura = spark.read.parquet('/dbfs/FileStore/tables/carga/usuarios')
dfleitura.show()

+------------+----------+-------------+--------------------+------+-------+
|PrimeiroNome|UltimoNome|Nacionalidade|           Profissao|Genero|Salario|
+------------+----------+-------------+--------------------+------+-------+
|       Giani|  Alcomini|     Italiano|Analista de sistemas|     M|   3000|
|        João|    Santos|   Brasileiro|           Estudante|     M|    500|
|       Peter|     Smith|    Americano|          Empresario|     M|  10000|
|       Pedro|    Coelho|    Portugues|           Professor|     M|   2500|
|       Carol|     Smith|    Americano|           Arquiteta|     F|   2000|
|      Krursa|  Zinovich|        Russa|              Medica|     F|   5000|
+------------+----------+-------------+--------------------+------+-------+



In [0]:
dfleitura.createOrReplaceTempView('Tabela_Parquet')
sql = spark.sql("select * from Tabela_Parquet where Salario>2000")
sql.show()

+------------+----------+-------------+--------------------+------+-------+
|PrimeiroNome|UltimoNome|Nacionalidade|           Profissao|Genero|Salario|
+------------+----------+-------------+--------------------+------+-------+
|       Giani|  Alcomini|     Italiano|Analista de sistemas|     M|   3000|
|       Peter|     Smith|    Americano|          Empresario|     M|  10000|
|       Pedro|    Coelho|    Portugues|           Professor|     M|   2500|
|      Krursa|  Zinovich|        Russa|              Medica|     F|   5000|
+------------+----------+-------------+--------------------+------+-------+



In [0]:
#Particionamento Parquet
dfparquet.write.partitionBy("Nacionalidade", "Profissao").mode("overwrite").parquet("/dbfs/FileStore/tables/carga/usuarios")

In [0]:
dfnacionalidade = spark.read.parquet("/dbfs/FileStore/tables/carga/usuarios/Nacionalidade=Americano")
dfnacionalidade.show(truncate=False)
#/dbfs/dbfs/FileStore/tables/carga/usuarios/Nacionalidade=Brasileiro

+------------+----------+------+-------+----------+
|PrimeiroNome|UltimoNome|Genero|Salario|Profissao |
+------------+----------+------+-------+----------+
|Carol       |Smith     |F     |2000   |Arquiteta |
|Peter       |Smith     |M     |10000  |Empresario|
+------------+----------+------+-------+----------+



In [0]:
spark.sql("CREATE OR REPLACE TEMPORARY VIEW Usuario USING parquet OPTIONS (path \"/dbfs/FileStore/tables/carga/usuarios/Nacionalidade=Americano\")")
spark.sql("select * from Usuario where UltimoNome = 'Smith'").show()

+------------+----------+------+-------+----------+
|PrimeiroNome|UltimoNome|Genero|Salario| Profissao|
+------------+----------+------+-------+----------+
|       Carol|     Smith|     F|   2000| Arquiteta|
|       Peter|     Smith|     M|  10000|Empresario|
+------------+----------+------+-------+----------+



Leitura de arquivo csv

In [0]:
df = spark.read.format("csv").option("header","true").load("/FileStore/tables/sleep_health.csv")
df.show()

+---------+------+---+--------------------+--------------+----------------+-----------------------+------------+-------------+--------------+----------+-----------+--------------+
|Person ID|Gender|Age|          Occupation|Sleep Duration|Quality of Sleep|Physical Activity Level|Stress Level| BMI Category|Blood Pressure|Heart Rate|Daily Steps|Sleep Disorder|
+---------+------+---+--------------------+--------------+----------------+-----------------------+------------+-------------+--------------+----------+-----------+--------------+
|        1|  Male| 27|   Software Engineer|           6.1|               6|                     42|           6|   Overweight|        126/83|        77|       4200|          None|
|        2|  Male| 28|              Doctor|           6.2|               6|                     60|           8|       Normal|        125/80|        75|      10000|          None|
|        3|  Male| 28|              Doctor|           6.2|               6|                     60| 

In [0]:
%scala
val dfs = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("/FileStore/tables/sleep_health.csv")
dfs.show()

+---------+------+---+--------------------+--------------+----------------+-----------------------+------------+-------------+--------------+----------+-----------+--------------+
Person ID|Gender|Age| Occupation|Sleep Duration|Quality of Sleep|Physical Activity Level|Stress Level| BMI Category|Blood Pressure|Heart Rate|Daily Steps|Sleep Disorder|
+---------+------+---+--------------------+--------------+----------------+-----------------------+------------+-------------+--------------+----------+-----------+--------------+
 1| Male| 27| Software Engineer| 6.1| 6| 42| 6| Overweight| 126/83| 77| 4200| None|
 2| Male| 28| Doctor| 6.2| 6| 60| 8| Normal| 125/80| 75| 10000| None|
 3| Male| 28| Doctor| 6.2| 6| 60| 8| Normal| 125/80| 75| 10000| None|
 4| Male| 28|Sales Representative| 5.9| 4| 30| 8| Obese| 140/90| 85| 3000| Sleep Apnea|
 5| Male| 28|Sales Representative| 5.9| 4| 30| 8| Obese| 140/90| 85| 3000| Sleep Apnea|
 6| Male| 28| Software Engineer| 5.9| 4| 30| 8| Obese| 140/90| 85| 3000| Insomnia|
 7| Male| 29| Teacher| 6.3| 6| 40| 7| Obese| 140/90| 82| 3500| Insomnia|
 8| Male| 29| Doctor| 7.8| 7| 75| 6| Normal| 120/80| 70| 8000| None|
 9| Male| 29| Doctor| 7.8| 7| 75| 6| Normal| 120/80| 70| 8000| None|
 10| Male| 29| Doctor| 7.8| 7| 75| 6| Normal| 120/80| 70| 8000| None|
 11| Male| 29| Doctor| 6.1| 6| 30| 8| Normal| 120/80| 70| 8000| None|
 12| Male| 29| Doctor| 7.8| 7| 75| 6| Normal| 120/80| 70| 8000| None|
 13| Male| 29| Doctor| 6.1| 6| 30| 8| Normal| 120/80| 70| 8000| None|
 14| Male| 29| Doctor| 6| 6| 30| 8| Normal| 120/80| 70| 8000| None|
 15| Male| 29| Doctor| 6| 6| 30| 8| Normal| 120/80| 70| 8000| None|
 16| Male| 29| Doctor| 6| 6| 30| 8| Normal| 120/80| 70| 8000| None|
 17|Female| 29| Nurse| 6.5| 5| 40| 7|Normal Weight| 132/87| 80| 4000| Sleep Apnea|
 18| Male| 29| Doctor| 6| 6| 30| 8| Normal| 120/80| 70| 8000| Sleep Apnea|
 19|Female| 29| Nurse| 6.5| 5| 40| 7|Normal Weight| 132/87| 80| 4000| Insomnia|
 20| Male| 30| Doctor| 7.6| 7| 75| 6| Normal| 120/80| 70| 8000| None|
+---------+------+---+--------------------+--------------+----------------+-----------------------+------------+-------------+--------------+----------+-----------+--------------+
only showing top 20 rows

dfs: org.apache.spark.sql.DataFrame = [Person ID: string, Gender: string ... 11 more fields]

In [0]:
df.write.parquet("/FileStore/tables/parquet_csv_sleep")

In [0]:
dfleitura=spark.read.parquet("/FileStore/tables/parquet_csv_sleep")
dfleitura.show()

+---------+------+---+--------------------+--------------+----------------+-----------------------+------------+-------------+--------------+----------+-----------+--------------+
|Person ID|Gender|Age|          Occupation|Sleep Duration|Quality of Sleep|Physical Activity Level|Stress Level| BMI Category|Blood Pressure|Heart Rate|Daily Steps|Sleep Disorder|
+---------+------+---+--------------------+--------------+----------------+-----------------------+------------+-------------+--------------+----------+-----------+--------------+
|        1|  Male| 27|   Software Engineer|           6.1|               6|                     42|           6|   Overweight|        126/83|        77|       4200|          None|
|        2|  Male| 28|              Doctor|           6.2|               6|                     60|           8|       Normal|        125/80|        75|      10000|          None|
|        3|  Male| 28|              Doctor|           6.2|               6|                     60| 